# Notebook for cleaning the electricity price data

Read data from the csv-files. Original data was xls-files, but there was something wrong with the file format. It seems that it was actually an HTML file and not a proper xls-file. We manually converted to xls-files in to csv-files in order to be able to read them.

In [1]:
import pandas as pd

prices_2016 = pd.read_csv('.\\data\\elspot-prices_2016_hourly_eur.csv', delimiter=';')
prices_2017 = pd.read_csv('.\\data\\elspot-prices_2017_hourly_eur.csv', delimiter=';')
prices_2018 = pd.read_csv('.\\data\\elspot-prices_2018_hourly_eur.csv', delimiter=';')
prices_2019 = pd.read_csv('.\\data\\elspot-prices_2019_hourly_eur.csv', delimiter=';')
prices_2020 = pd.read_csv('.\\data\\elspot-prices_2020_hourly_eur.csv', delimiter=';')
prices_2021 = pd.read_csv('.\\data\\elspot-prices_2021_hourly_eur.csv', delimiter=';')

print(prices_2016.head(1))
print(prices_2017.head(1))
print(prices_2018.head(1))
print(prices_2019.head(1))
print(prices_2020.head(1))
print(prices_2021.head(1))


  Unnamed: 0    Hours    SYS    SE1    SE2    SE3    SE4     FI    DK1    DK2  \
0   1.1.2016  00�-�01  16,39  16,39  16,39  16,39  16,39  16,39  16,39  16,39   

    Oslo Kr.sand Bergen  Molde Tr.heim Troms�     EE     LV     LT  FRE  
0  16,39   16,39  16,39  16,39   16,39  16,39  16,39  28,11  28,11  NaN  
  Unnamed: 0    Hours   SYS    SE1    SE2    SE3    SE4     FI    DK1    DK2  \
0   1.1.2017  00�-�01  25,7  24,03  24,03  24,03  24,03  24,03  20,96  20,96   

   Oslo Kr.sand Bergen  Molde Tr.heim Troms�     EE     LV     LT  
0  28,4    28,4   28,4  24,03   24,03  24,03  24,03  24,03  24,03  
  Unnamed: 0    Hours    SYS    SE1    SE2    SE3    SE4     FI   DK1    DK2  \
0   1.1.2018  00�-�01  26,31  26,33  26,33  26,33  26,33  26,33  21,8  26,33   

    Oslo Kr.sand Bergen  Molde Tr.heim Troms�     EE     LV     LT  
0  26,33   26,33  26,33  26,33   26,33  26,33  26,33  26,33  26,33  
  Unnamed: 0    Hours    SYS    SE1    SE2    SE3    SE4     FI    DK1    DK2  \
0   1.1.2019

Keep datetime information and filter out other than Finnish price data.

In [2]:
from datetime import datetime as dt

# rename date information column
prices_2016 = prices_2016.rename(columns={'Unnamed: 0':'Date'})
prices_2017 = prices_2017.rename(columns={'Unnamed: 0':'Date'})
prices_2018 = prices_2018.rename(columns={'Unnamed: 0':'Date'})
prices_2019 = prices_2019.rename(columns={'Unnamed: 0':'Date'})
prices_2020 = prices_2020.rename(columns={'Unnamed: 0':'Date'})
prices_2021 = prices_2021.rename(columns={'Unnamed: 0':'Date'})


# keep only columns that are needed
prices_2016 = prices_2016[['Date', 'Hours', 'FI']]
prices_2017 = prices_2017[['Date', 'Hours', 'FI']]
prices_2018 = prices_2018[['Date', 'Hours', 'FI']]
prices_2019 = prices_2019[['Date', 'Hours', 'FI']]
prices_2020 = prices_2020[['Date', 'Hours', 'FI']]
prices_2021 = prices_2021[['Date', 'Hours', 'FI']]

# concat data
df = pd.concat([
    prices_2016, 
    prices_2017, 
    prices_2018, 
    prices_2019, 
    prices_2020, 
    prices_2021]
)

# rename consumption column
df = df.rename(columns={'FI':'PRICE (EUR/MWh)'})

# drop rows where date is more than 31.8.2021
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y')
end = dt.strptime('01.09.2021', '%d.%m.%Y')
df = df[df['Date'] < end]

print(df.head(1))
print(df.shape)

        Date    Hours PRICE (EUR/MWh)
0 2016-01-01  00�-�01           16,39
(49685, 3)


Clean the Hours column data. The data is in hh-hh format, but we would like the format in 0-23. So we only need the starting hour of the one hour time interval.

In [3]:
try:
    df['Hours'] = df['Hours'].map(lambda hours_str: int(hours_str[0:2]))
    df = df.rename(columns={'Hours':'Hour'})
except:
    pass
    
print(df.head(1))
print(df.shape)

        Date  Hour PRICE (EUR/MWh)
0 2016-01-01     0           16,39
(49685, 3)


Write the clean data into csv-file.

In [4]:
df.to_csv('electricity-prices-FI_2016-2021_hourly.csv', sep=';', encoding='utf-8', index=False)
print('Success')

Success
